In [1]:
using Tetris
include("../core/TetrisAICore.jl")
import .TetrisAICore: loadmodel, predict, Model,
 GameState, MoveState, Action, valid_movement, 
 move, put_mino!, generate_minopos, get_node_list,
  Node, draw_game, init_screen, get_key_state, mysleep,
mino_to_array, AILux, create_model, select_node, vector2array
using CUDA
using JLD2
using HTTP
using CodecZstd
using Serialization
CUDA.math_mode!(CUDA.PEDANTIC_MATH)
module TetrisAICore
module AILux
using CUDA
using Lux, LuxCUDA, NNlib, MLUtils, Zygote
import Lux: gpu_device, cpu_device
gpu = gpu_device()
cpu = cpu_device()
export gpu, cpu
using JLD2, Optimisers
using Statistics, Random
using NamedTupleTools
include("play/layers.jl")
include("play/network.jl")
export QNetwork
end
using .AILux
export QNetwork, gpu, cpu
end
using .TetrisAICore

┌ Info: CUDA is on
└ @ Main.TetrisAICore.AILux c:\work_space\programing\julia\TetrisAI\src\core\ai\lux\Lux.jl:13


In [2]:
model, ps, st = loadmodel("play/mainmodel.jld2")

(_QNetwork(), (board_net = (conv1 = (weight = Float32[-0.0012234598 -0.0029251971 -0.050272558; 0.0014352337 0.025611553 0.012174988; 0.036551222 0.057714533 0.045742106;;; -0.012313262 -0.04012356 0.00768004; -0.038832806 0.043806687 -0.022630272; 0.035085622 0.06955569 0.032398745;;;; -0.04290273 0.046842184 0.0385517; 0.010850708 0.03335503 0.010544146; -0.022736242 -0.04411315 -0.0186169;;; -0.021284552 0.008136403 0.037287712; 0.028977877 0.025845481 0.0104257185; -0.061901927 0.02318343 -0.0585937;;;; 0.005364301 0.041104533 -0.00087841036; -0.008213836 -0.06817173 0.020378724; 0.014750741 -0.02324493 0.035116866;;; -0.0074012694 0.058979705 -0.01898393; -0.035696045 0.00050721853 -0.013690126; -0.013687363 0.03920199 -0.038921803;;;; … ;;;; -0.0309638 -0.0105528915 -0.0047275745; 0.02730028 0.0073370137 -0.051424496; 0.048876245 0.023851847 0.014525884;;; -0.019214176 0.07819214 -0.02147594; -0.0053820233 0.026012434 0.027075432; 0.050644815 0.044418305 -0.025095819;;;; 0.015536

In [3]:
model

_QNetwork(
    board_net = BoardNet(
        conv1 = Conv((3, 3), 2 => 256, pad=1),  # 4_864 parameters
        norm1 = BatchNorm(256, affine=true, track_stats=true),  # 512 parameters, plus 513
        resblocks = Chain(
            layer_1 = SkipConnection(
                Chain(
                    layer_1 = Conv((3, 3), 256 => 256, pad=1),  # 590_080 parameters
                    layer_2 = BatchNorm(256, affine=true, track_stats=true),  # 512 parameters, plus 513
                    layer_3 = WrappedFunction(swish),
                    layer_4 = Conv((3, 3), 256 => 256, pad=1),  # 590_080 parameters
                    layer_5 = BatchNorm(256, affine=true, track_stats=true),  # 512 parameters, plus 513
                    layer_6 = SkipConnection(
                        Chain(
                            layer_1 = GlobalMeanPool(),
                            layer_2 = Conv((1, 1), 256 => 64, swish),  # 16_448 parameters
                            layer_3 = Conv((1, 1), 64 => 25

In [4]:
# 24x10
board = [
    0 0 0 0 0 0 0 0 0 0
    0 0 0 0 0 0 0 0 0 0
    0 0 0 0 0 0 0 0 0 0
    0 0 0 0 0 0 0 0 0 0
    0 0 0 0 0 0 0 0 0 0
    0 0 0 0 0 0 0 0 0 0
    0 0 0 0 0 0 0 0 0 0
    0 0 0 0 0 0 0 0 0 0
    0 0 0 0 0 0 0 0 0 0
    0 0 0 0 0 0 0 0 0 0
    0 0 0 0 0 0 0 0 0 0
    0 0 0 0 0 0 0 0 0 0
    0 0 0 0 0 0 0 0 0 0
    0 0 0 0 0 0 0 0 0 0
    0 0 0 0 0 0 0 0 0 0
    0 0 0 0 0 0 0 0 0 0
    0 0 0 0 0 0 0 0 0 0
    0 0 0 0 0 0 0 0 0 0
    1 1 1 1 0 0 0 0 0 0
    1 1 1 1 1 0 0 0 0 0
    1 1 1 1 1 1 0 0 0 0 
    1 1 1 1 1 1 1 0 1 0 
    1 1 1 1 1 1 1 1 1 0 
    1 1 1 1 1 1 1 1 1 0    
]
nothing

In [5]:
# 24x10
minopos = [
    0 0 0 0 0 0 0 0 0 0
    0 0 0 0 0 0 0 0 0 0
    0 0 0 0 0 0 0 0 0 0
    0 0 0 0 0 0 0 0 0 0
    0 0 0 0 0 0 0 0 0 0
    0 0 0 0 0 0 0 0 0 0
    0 0 0 0 0 0 0 0 0 0
    0 0 0 0 0 0 0 0 0 0
    0 0 0 0 0 0 0 0 0 0
    0 0 0 0 0 0 0 0 0 0
    0 0 0 0 0 0 0 0 0 0
    0 0 0 0 0 0 0 0 0 0
    0 0 0 0 0 0 0 0 0 0
    0 0 0 0 0 0 0 0 0 0
    0 0 0 0 0 0 0 0 0 0
    0 0 0 0 0 0 0 0 0 0
    0 0 0 0 0 0 0 0 0 0
    0 0 0 0 0 0 0 0 0 0
    0 0 0 0 0 0 0 0 0 0
    0 0 0 0 0 0 0 0 0 0
    0 0 0 0 0 0 1 1 1 0
    0 0 0 0 0 0 0 1 0 0
    0 0 0 0 0 0 0 0 0 0
    0 0 0 0 0 0 0 0 0 0
]
nothing

In [6]:
for i in 2:7, j in 2:7, k in 2:7, l in 2:7, m in 2:7
    length(Set([i, j, k, l, m])) != 5 && continue
    combo = 0
    btb = 0
    tspin = 0
    # [i_mino, o_mino, s_mino, z_mino, j_mino, l_mino, t_mino]
    holdnext = zeros(7, 6)
    holdnext[j, 1] = 1
    holdnext[k, 2] = 1
    holdnext[l, 3] = 1
    holdnext[m, 4] = 1
    holdnext[i, 5] = 1
    holdnext[1, 6] = 1
    board_array = reshape(board, (24, 10, 1, 1))
    minopos_array = reshape(minopos, (24, 10, 1, 1))
    combo_array = reshape([combo], (1, 1))
    btb_array = reshape([btb], (1, 1))
    tspin_array = reshape([tspin], (1, 1))
    holdnext_array = reshape(holdnext, (7, 6, 1))
    r, _ = model((board_array, minopos_array, combo_array, btb_array, tspin_array, holdnext_array), ps, st)
    println(r[1])
end
nothing

7.472570421928223


7.530180516121118


7.484526685120707


7.530803768262506


7.549182031685069


...

7.509722682218667


7.5150090142839


7.522312176263714


7.507863921482549


7.525519377862906


In [24]:
# 24x10
board = [
    0 0 0 0 0 0 0 0 0 0
    0 0 0 0 0 0 0 0 0 0
    0 0 0 0 0 0 0 0 0 0
    0 0 0 0 0 0 0 0 0 0
    0 0 0 0 0 0 0 0 0 0
    0 0 0 0 0 0 0 0 0 0
    0 0 0 0 0 0 0 0 0 0
    0 0 0 0 0 0 0 0 0 0
    0 0 0 0 0 0 0 0 0 0
    0 0 0 0 0 0 0 0 0 0
    0 0 0 0 0 0 0 0 0 0
    0 0 0 0 0 0 0 0 0 0
    0 0 0 0 0 0 0 0 0 0
    0 0 0 0 0 0 0 0 0 0
    0 0 0 0 0 0 0 0 0 0
    0 0 0 0 0 0 0 0 0 0
    0 0 0 0 0 0 0 0 0 0
    0 0 0 0 0 0 0 0 0 0
    1 1 1 1 0 0 0 0 0 0
    1 1 1 1 1 0 0 0 0 0
    1 1 1 1 1 1 0 0 0 0 
    1 1 1 1 1 1 1 0 1 0 
    1 1 1 1 1 1 1 1 1 0 
    1 1 1 1 1 1 1 1 1 0    
]
nothing

In [25]:
# 24x10
minopos = [
    0 0 0 0 0 0 0 0 0 0
    0 0 0 0 0 0 0 0 0 0
    0 0 0 0 0 0 0 0 0 0
    0 0 0 0 0 0 0 0 0 0
    0 0 0 0 0 0 0 0 0 0
    0 0 0 0 0 0 0 0 0 0
    0 0 0 0 0 0 0 0 0 0
    0 0 0 0 0 0 0 0 0 0
    0 0 0 0 0 0 0 0 0 0
    0 0 0 0 0 0 0 0 0 0
    0 0 0 0 0 0 0 0 0 0
    0 0 0 0 0 0 0 0 0 0
    0 0 0 0 0 0 0 0 0 0
    0 0 0 0 0 0 0 0 0 0
    0 0 0 0 0 0 0 0 0 0
    0 0 0 0 0 0 0 0 0 0
    1 1 0 0 0 0 0 0 0 0
    1 1 0 0 0 0 0 0 0 0
    0 0 0 0 0 0 0 0 0 0
    0 0 0 0 0 0 0 0 0 0
    0 0 0 0 0 0 0 0 0 0
    0 0 0 0 0 0 0 0 0 0
    0 0 0 0 0 0 0 0 0 0
    0 0 0 0 0 0 0 0 0 0
]
nothing

In [33]:
target = 6
hold = 1
for i in 1:7, j in 1:7, k in 1:7, l in 1:7
    length(Set([i, j, k, l])) != 4 && continue
    target ==i || target == j || target == k || target == l && continue
    combo = 0
    btb = 0
    tspin = 0
    # [i_mino, o_mino, s_mino, z_mino, j_mino, l_mino, t_mino]
    holdnext = zeros(7, 6)
    holdnext[hold, 1] = 1
    holdnext[i, 2] = 1
    holdnext[j, 3] = 1
    holdnext[k, 4] = 1
    holdnext[l, 5] = 1
    holdnext[target, 6] = 1
    board_array = reshape(board, (24, 10, 1, 1))
    minopos_array = reshape(minopos, (24, 10, 1, 1))
    combo_array = reshape([combo], (1, 1))
    btb_array = reshape([btb], (1, 1))
    tspin_array = reshape([tspin], (1, 1))
    holdnext_array = reshape(holdnext, (7, 6, 1))
    r, _ = model((board_array, minopos_array, combo_array, btb_array, tspin_array, holdnext_array), ps, st)
    println(r[1])
end
nothing

7.915154907025874


7.930878019198616


7.919324588339258


7.912712576640037


7.924294638250374


...

7.978569129627873


7.990104677543372


7.968189826410949


7.980003471794598


7.976271645467913


In [3]:
# 24x10
board = [
    0 0 0 0 0 0 0 0 0 0
    0 0 0 0 0 0 0 0 0 0
    0 0 0 0 0 0 0 0 0 0
    0 0 0 0 0 0 0 0 0 0
    0 0 0 0 0 0 0 0 0 0
    0 0 0 0 0 0 0 0 0 0
    0 0 0 0 0 0 0 0 0 0
    0 0 0 0 0 0 0 0 0 0
    0 0 0 0 0 0 0 0 0 0
    0 0 0 0 0 0 0 0 0 0
    0 0 0 0 0 0 0 0 0 0
    0 0 0 0 0 0 0 0 0 0
    0 0 0 0 0 0 0 0 0 0
    0 0 0 0 0 0 0 0 0 0
    0 0 0 0 0 0 0 0 0 0
    0 0 0 0 0 0 0 0 0 0
    0 0 0 0 0 0 0 0 0 0
    0 0 0 0 0 0 0 0 0 0
    1 1 1 1 0 0 0 0 0 0
    1 1 1 1 1 0 0 0 0 0
    1 1 1 1 1 1 0 0 0 0 
    1 1 1 1 1 1 1 0 0 0 
    1 1 1 1 1 1 0 0 0 0 
    1 1 1 1 1 1 1 0 0 0    
]
nothing

In [4]:
# 24x10
minopos = [
    0 0 0 0 0 0 0 0 0 0
    0 0 0 0 0 0 0 0 0 0
    0 0 0 0 0 0 0 0 0 0
    0 0 0 0 0 0 0 0 0 0
    0 0 0 0 0 0 0 0 0 0
    0 0 0 0 0 0 0 0 0 0
    0 0 0 0 0 0 0 0 0 0
    0 0 0 0 0 0 0 0 0 0
    0 0 0 0 0 0 0 0 0 0
    0 0 0 0 0 0 0 0 0 0
    0 0 0 0 0 0 0 0 0 0
    0 0 0 0 0 0 0 0 0 0
    0 0 0 0 0 0 0 0 0 0
    0 0 0 0 0 0 0 0 0 0
    0 0 0 0 0 0 0 0 0 0
    0 0 0 0 0 0 0 0 0 0
    1 1 0 0 0 0 0 0 0 0
    1 1 0 0 0 0 0 0 0 0
    0 0 0 0 0 0 0 0 0 0
    0 0 0 0 0 0 0 0 0 0
    0 0 0 0 0 0 0 0 0 0
    0 0 0 0 0 0 0 0 0 0
    0 0 0 0 0 0 0 0 0 0
    0 0 0 0 0 0 0 0 0 0
]
nothing

In [14]:
target = 5
hold = 5 
for i in 1:7, j in 1:7, k in 1:7, l in 1:7
    length(Set([i, j, k, l])) != 4 && continue
    target ==i || target == j || target == k || target == l && continue
    combo = 0
    btb = 1
    tspin = 0
    # [i_mino, o_mino, s_mino, z_mino, j_mino, l_mino, t_mino]
    holdnext = zeros(7, 6)
    holdnext[hold, 1] = 1
    holdnext[i, 2] = 1
    holdnext[j, 3] = 1
    holdnext[k, 4] = 1
    holdnext[l, 5] = 1
    holdnext[target, 6] = 1
    board_array = reshape(board, (24, 10, 1, 1))
    minopos_array = reshape(minopos, (24, 10, 1, 1))
    combo_array = reshape([combo], (1, 1))
    btb_array = reshape([btb], (1, 1))
    tspin_array = reshape([tspin], (1, 1))
    holdnext_array = reshape(holdnext, (7, 6, 1))
    r, _ = model((board_array, minopos_array, combo_array, btb_array, tspin_array, holdnext_array), ps, st)
    println(r[1])
end
nothing

7.85464446510068


7.856027094852622


7.82546824638616


7.853745429273223


7.852663220526766


...

7.89516741572347


7.9283702035806405


7.899250853079877


7.907810017825485


7.906354422671662
